In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
# import openpyxl
from openpyxl import load_workbook

In [16]:

wb = load_workbook("datasets/hindi.xlsx")
sheet = wb.active
col = csv.writer(open("tt.csv", 'w', newline=""))
for r in sheet.rows:
    col.writerow([cell.value for cell in r])
  
df = pd.DataFrame(pd.read_csv("tt.csv"))
df.to_csv("datasets/hindi.csv", sep=",")
df.head(5)

,text,task1
0,1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n...,HATE
1,"RT @Vishesh4: @jawaharyadavbjp जवाहर यादव, अगर...",NOT_HATE
2,RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही ...,HATE
3,Yey nina khothani labafazi benu phambili Finis...,HATE
4,RT @Rajeshbhanjan2: जब भी कोई सिकुलर कोंग्रेसी...,HATE


In [21]:
df['text'] = df['text'].map(lambda x: x.lstrip('RT@').rstrip('RT@'))
df.head()

,text,task1
0,1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n...,HATE
1,"@Vishesh4: @jawaharyadavbjp जवाहर यादव, अगर ह...",NOT_HATE
2,@FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही कर...,HATE
3,Yey nina khothani labafazi benu phambili Finis...,HATE
4,@Rajeshbhanjan2: जब भी कोई सिकुलर कोंग्रेसी ग...,HATE


In [22]:
df['task1'] = df['task1'].map({'HATE':1, 'NOT_HATE':0})
df.head()
print(df["text"][2])

 @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही कर सकते लेकिन बुरखा पहन के कर सकते हैं @yadavakhilesh ?

ये आज़मगढ़ है और यहाँ से "टोटी चोर"…


In [4]:
import collections

with open("datasets/Hindi_StopWords.txt",encoding='utf-8') as f:
    stopword= f.read().strip('\ufeff')
stopword = stopword.split(", ")
stopword = [i.strip("'") for i in stopword]

stopwords = set(stopword)
print(stopwords)

{'इसकी', 'कई', 'इन', 'बिलकुल', "इतयादि' ,'यहाँ", 'अप', 'द्वारा', 'ना', 'बाद', 'हे', 'तिस', 'कहते', 'जिन्हों', 'किस', 'कौन', 'होता', 'सारा', 'वहिं', 'कहा', 'घर', 'फिर', 'उनको', 'दूसरे', 'हुइ', 'अर्थात', 'ने', 'तुझे', 'क्योंकि', 'उनका', 'दबारा', 'का', 'मानो', 'अत', 'जैसा', 'ओर', 'उसी', 'निचे', 'वुह', 'वह ', 'लिये', 'तब', 'पहले', 'नहिं', 'उनकि', 'एक', 'किसि', 'यिह', 'किन्हें', 'हम', 'साबुत', 'वुह ', 'जेसा', 'रह', 'संग', 'मुझ', 'उनकी', 'ऱ्वासा', 'ही', 'एस', 'सभी', 'वगेरह', 'एसे', 'हुअ', 'तुम', 'यदि', 'जितना', 'कौनसा', 'यहि', 'करता', 'या', 'मेरी', 'वहां', 'अदि', 'तेरा', 'पर  ', 'इनका', 'और', 'साथ', 'तेरे', 'तिसे', 'किसे', 'जा', 'उसने', 'न', 'इसकि', 'तिन्हों', 'तेरी', 'उन्ह', 'होने', 'अपना', 'होना', 'रहे', 'किया', 'जिंहें', 'इंहिं', 'गए', 'जिसे', 'हें', 'पूरा', 'वह', 'उन्हीं', 'अपनि', 'पे', 'उसकी', 'तक', 'काफि', 'दुसरे', 'अपनी', 'होते', 'उसे', 'इसलिये', 'किंहें', 'तिंहें', 'कह', 'इत्यादि', 'यह', 'कोन', 'मैं', 'मगर', 'जो', 'थी', 'उसका', 'इन्हें', 'उन', 'कोनसा', 'भितर', 'उंहों', 'कइ', 'ऐसे', '

In [32]:
import itertools

hate_det = df[df.task1==1]
hate_det.reset_index(drop=True, inplace=True)
acc_det = df[df.task1==0]
acc_det.reset_index(drop=True, inplace=True)

# Tokenization
hate_news = []
for rows in range(0, hate_det.shape[0]):
    head_txt = hate_det.text[rows]
    head_txt = head_txt.split(" ")
    hate_news.append(head_txt)

hate_list = list(itertools.chain(*hate_news))
print(hate_list[1:20])
print("\n" + "Number of words: " + str(len(hate_list)))
acc_news = []
for rows in range(0, acc_det.shape[0]):
    head_txt = acc_det.text[rows]
    head_txt = head_txt.split(" ")
    acc_news.append(head_txt)

acc_list = list(itertools.chain(*acc_news))
print("\n" + str(acc_list[1:20]))
print("\n" + "Number of words: " + str(len(acc_list)))

['आदमीं', 'को', 'मारने', 'पर', 'गोडसे', 'आतंकी', 'हो', 'सके', 'है', 'तो\n17000', 'सिखो,', '5000', 'भोपाली,', '3000', 'तमिलों', 'का', 'कत्लेआम', 'करवाने', 'वाला']

Number of words: 20806

['@Vishesh4:', '@jawaharyadavbjp', 'जवाहर', 'यादव,', 'अगर', 'हिम्मत', 'है', 'तो', 'पूरा', 'वीडियो', 'डालो', 'फिर', 'तुम्हारे', 'झूठ', 'सामने', 'आएंगे।\nदीपेंदर', 'ने', 'साफ', 'कहा,']

Number of words: 46970


### Using CNN model

In [7]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import accuracy_score

In [24]:
X = df.text
Y = df.task1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

In [25]:
tk = tf.keras.preprocessing.text.Tokenizer(num_words=1000)
tk.fit_on_texts(X_train)
seqs = tk.texts_to_sequences(X_train)
max_len = 100
seqs_mat = tf.keras.preprocessing.sequence.pad_sequences(seqs,maxlen=max_len)
X_train[1:5]

1364    BKaurDeep मोदी ने IPL का कप धोनी से छीन कर अम्...
2898     @ViShNuKeer18: #Allah_Kabir\nकबीर जी ने समझाय...
216     Jitendr14272284 @Pinkimishr क्या आपने किसी मुस...
786      @MKS1703: इस बन्दे ने साबित कर दिया कि इंसानि...
Name: text, dtype: object

In [26]:
def model_def():
    inputs = tf.keras.layers.Input(name='inputs', shape=[max_len])
    layer = tf.keras.layers.Embedding(1000,50,input_length=max_len)(inputs)
    layer = tf.keras.layers.LSTM(64)(layer)
    layer = tf.keras.layers.Dense(256,name='FC1')(layer)
    layer = tf.keras.layers.Activation('relu')(layer)
    layer = tf.keras.layers.Dropout(0.2)(layer)
    layer = tf.keras.layers.Dense(1,name='out_layer')(layer)
    
    model = tf.keras.models.Model(inputs=inputs,outputs=layer)
    return model

In [27]:
model = model_def()
model.summary()
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 100)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 50)           50000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257 

In [28]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_acc', 
    mode='max',
    patience=10
)
# increase epochs and other steps for better training
model.fit(seqs_mat,Y_train,batch_size=100,epochs=25,
          validation_split=0.1,callbacks=[es])

Train on 2133 samples, validate on 237 samples
Epoch 1/25
2133/2133 [==============================] - 8s 4ms/sample - loss: 0.7438 - acc: 0.7037 - val_loss: 0.5521 - val_acc: 0.7511
Epoch 2/25
2133/2133 [==============================] - 6s 3ms/sample - loss: 0.5418 - acc: 0.7623 - val_loss: 1.0164 - val_acc: 0.7468
Epoch 3/25
2133/2133 [==============================] - 6s 3ms/sample - loss: 0.5108 - acc: 0.8186 - val_loss: 1.4022 - val_acc: 0.7426
Epoch 4/25
2133/2133 [==============================] - 6s 3ms/sample - loss: 0.4598 - acc: 0.8594 - val_loss: 1.7488 - val_acc: 0.7384
Epoch 5/25
2133/2133 [==============================] - 6s 3ms/sample - loss: 0.4048 - acc: 0.8748 - val_loss: 2.1271 - val_acc: 0.7257
Epoch 6/25
2133/2133 [==============================] - 6s 3ms/sample - loss: 0.3807 - acc: 0.9067 - val_loss: 2.4914 - val_acc: 0.7215
Epoch 7/25
2133/2133 [==============================] - 6s 3ms/sample - loss: 0.4842 - acc: 0.9058 - val_loss: 2.8704 - val_acc: 0.7004
E

In [29]:
test_sequences = tk.texts_to_sequences(X_test)
test_sequences_matrix = tf.keras.preprocessing.sequence.pad_sequences(test_sequences,maxlen=max_len)

accr = model.evaluate(test_sequences_matrix,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

593/593 [==============================] - 1s 1ms/sample - loss: 1.1266 - acc: 0.7083
Test set
  Loss: 1.127
  Accuracy: 0.708


In [30]:
def predict_hate(user_seq):
#     prediction
    prob = model.predict(user_seq)
    probability = np.mean(prob, axis=0)

    if probability > 0.5:
        return("Hate")
    elif probability < 0.5:
        return("Not Hate")
    elif probability == 0.5:
        return("Neutral")

def user_text_processing(user_text):
    user_text = user_text.split()
    user_text = [word.lower() for word in user_text if word not in stopwords]
#     user_text = [lemm.lemmatize(word) for word in user_text]
    user_text
    user_seq = np.array(user_text)
    user_seq = tk.texts_to_sequences(user_seq)
    user_seq = tf.keras.preprocessing.sequence.pad_sequences(user_seq,maxlen=max_len)

    return user_seq

In [31]:
user_text = 'RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही कर सकते लेकिन बुरखा पहन के कर सकते हैं @yadavakhilesh ? ये आज़मगढ़ है और यहाँ से "टोटी चोर"…'
user_seq = user_text_processing(user_text)
user_seq
prediction = predict_hate(user_seq)
print(f"Sentence '{user_text}' is of '{prediction}' nature")


Sentence 'RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही कर सकते लेकिन बुरखा पहन के कर सकते हैं @yadavakhilesh ? ये आज़मगढ़ है और यहाँ से "टोटी चोर"…' is of 'Not Hate' nature


### Using Logistic regression

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

train_texts, valid_texts, y_train, y_valid = train_test_split(df['text'], df['task1'], random_state=17)

In [37]:
tf_idf = TfidfVectorizer(ngram_range=(1, 2), max_features=50000, min_df=2)

logit = LogisticRegression(C=1, n_jobs=4, solver='lbfgs', random_state=17, verbose=1)
tfidf_logit_pipeline = Pipeline([('tf_idf', tf_idf), ('logit', logit)])

In [39]:
%%time
tfidf_logit_pipeline.fit(train_texts, y_train)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


CPU times: user 369 ms, sys: 3.68 ms, total: 372 ms
Wall time: 1.88 s


[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    1.5s finished


Pipeline(steps=[('tf_idf',
                 TfidfVectorizer(max_features=50000, min_df=2,
                                 ngram_range=(1, 2))),
                ('logit',
                 LogisticRegression(C=1, n_jobs=4, random_state=17,
                                    verbose=1))])

In [41]:
%%time
valid_pred = tfidf_logit_pipeline.predict(valid_texts)

CPU times: user 75.4 ms, sys: 0 ns, total: 75.4 ms
Wall time: 77.4 ms


In [42]:
accuracy_score(y_valid, valid_pred)

0.7139001349527665

In [44]:
import eli5
eli5.show_weights(estimator=tfidf_logit_pipeline.named_steps['logit'], vec=tfidf_logit_pipeline.named_steps['tf_idf'])

ModuleNotFoundError: No module named 'eli5'